In [56]:

from collections import defaultdict
import qiskit
from qiskit.providers.aer import PulseSimulator
from qiskit.providers.aer.pulse import PulseSystemModel
import numpy as np

from qiskit import IBMQ, pulse, QuantumCircuit, QuantumRegister, ClassicalRegister, assemble
from qiskit.providers.aer.pulse.hamiltonian_model import HamiltonianModel
from qiskit.quantum_info import Operator, Pauli
from qiskit.tools.monitor import job_monitor
from qutip import Qobj
from src.qoc_instruction_schedule_map import QOCInstructionScheduleMap
from qiskit.test.mock import FakeArmonk, FakeAthens, FakeAlmaden, FakeValencia
from src.qutip_helper import convert_qutip_ham
from src.QutipOptimizer import QutipOptimizer, prefactor_parser

In [1]:

IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
real_backend = provider.get_backend('ibmq_athens')
# backend=FakeArmonk()
backend=FakeValencia()

NameError: name 'IBMQ' is not defined

In [58]:
real_backend.defaults().qubit_freq_est

[5175596479.305874,
 5267841583.026349,
 5051865353.594587,
 4856353661.276202,
 5118587010.330171]

In [59]:
# convert_qutip_ham(real_backend, False, [0])

In [60]:
num_qub=5
lst = ['omegad' + str(i) for i in range(num_qub)]
for ele in lst:
    getattr(backend.configuration(), 'hamiltonian')['vars'][ele] = 0.5 * 1e9 * getattr(real_backend.configuration(), 'hamiltonian')['vars'][ele]
# freq_est = 4.97e9
# armonk_backend.defaults().qubit_freq_est = [freq_est]
lst2 = ['wq' + str(i) for i in range(num_qub)]
for i,ele in enumerate(lst2):
    getattr(backend.configuration(), 'hamiltonian')['vars'][ele] = 2 * np.pi * real_backend.defaults().qubit_freq_est[i]

In [61]:
backend.defaults().qubit_freq_est = real_backend.defaults().qubit_freq_est

In [62]:
circ = QuantumCircuit(num_qub)

circ.x(0)
# circ.x(2)
circ.measure_all()


In [63]:
# def_trans_circ = qiskit.transpile(circ, basis_gates=backend.configuration().basis_gates)
# default_schedule = qiskit.schedule(def_trans_circ, inst_map = backend.defaults().instruction_schedule_map,
#                         meas_map = backend.configuration().meas_map)

In [64]:
# backend_sim = PulseSimulator(configuration=backend.configuration)
backend_sim = PulseSimulator()

pulse_model = PulseSystemModel.from_backend(backend)

In [65]:
# # # def_trans_circ = qiskit.transpile(circ, basis_gates=backend.configuration().basis_gates)
# # default_schedule = qiskit.schedule(def_trans_circ, inst_map = backend.defaults().instruction_schedule_map,
#                         meas_map = backend.configuration().meas_map)

#
# # omegad0 = 31919806.545849085
# # getattr(backend.configuration(), 'hamiltonian')['vars']['omegad0'] = omegad0
# # getattr(backend.configuration(), 'hamiltonian')['vars']['wq0'] = 2*np.pi*freq_est
# default_qobj = assemble(default_schedule,
#                       backend=backend_sim,
#                       qubit_lo_freq=backend.defaults().qubit_freq_est,
#                       meas_level=2,
#                       meas_return='single',
#                       shots=1024)

In [66]:
# sim_result = backend_sim.run(grape_qobj, pulse_model).result()
# def_sim_result = backend_sim.run(default_qobj, pulse_model).result()
# print(sim_result.get_counts())
# print(def_sim_result.get_counts())

In [67]:
# def_sim_result.get_counts(0)
# def_sim_result.get_counts()

In [68]:
grape_optimizer = QutipOptimizer(backend, n_ts=800)

In [69]:
builtin_instructions = backend.defaults().instruction_schedule_map

In [70]:
qoc_inst_map = QOCInstructionScheduleMap.from_inst_map(grape_optimizer,builtin_instructions)

In [71]:
grape_schedule = qiskit.schedule(circ, inst_map = qoc_inst_map,
                        meas_map = backend.configuration().meas_map)

INFO:qutip.control.dynamics:Setting memory optimisations for level 0
INFO:qutip.control.dynamics:Internal operator data type choosen to be <class 'numpy.ndarray'>
INFO:qutip.control.dynamics:phased dynamics generator caching True
INFO:qutip.control.dynamics:propagator gradient caching True
INFO:qutip.control.dynamics:eigenvector adjoint caching True
INFO:qutip.control.dynamics:use sparse eigen decomp False
INFO:qutip.control.pulseoptim:System configuration:
Drift Hamiltonian:
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 1.62596159e+10  0.00000000e+00]
 [ 0.00000000e+00 -1.62596159e+10]]
Control 1 Hamiltonian:
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.00000000e+00 2.38947225e+08]
 [2.38947225e+08 0.00000000e+00]]
Control 2 Hamiltonian:
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.+0.00000000e+00j 0.-2.38947225e+08j]
 [0.+2.38947225e+08j 0.+

88
34
[Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.00000000e+00 2.38947225e+08]
 [2.38947225e+08 0.00000000e+00]], Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.+0.00000000e+00j 0.-2.38947225e+08j]
 [0.+2.38947225e+08j 0.+0.00000000e+00j]]]
45
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]
[Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.00000000e+00 2.38947225e+08]
 [2.38947225e+08 0.00000000e+00]], Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.+0.00000000e+00j 0.-2.38947225e+08j]
 [0.+2.38947225e+08j 0.+0.00000000e+00j]]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 1.26486811e-05+2.52501229e-06j -1.15668850e-05-1.00000000e+00j]
 [ 1.01363736e-05-1.00000000e+00j  1.26486775e-05-2.525030

In [72]:

grape_qobj = assemble(grape_schedule,
                      backend=backend,
                      qubit_lo_freq=backend.defaults().qubit_freq_est,
                      meas_level=2,
                      meas_return='single',
                      shots=4096)

In [73]:
sim_result = backend_sim.run(grape_qobj, pulse_model).result()
sim_result.get_counts()

{'00000': 2623, '00001': 1473}

In [74]:
job = real_backend.run(grape_qobj)#, num_shots=4096)

In [75]:
job_monitor(job)

Job Status: job incurred error     


In [76]:
# def_job = backend.run(default_qobj, num_shots=4096)

In [77]:
job.result().get_counts()

IBMQJobFailureError: 'Unable to retrieve result for job 5f3e78645133a70013de671f. Job has failed. Use job.error_message() to get more details.'

In [ ]:
# def_job.result().get_counts()

In [ ]:
# valencia_job = job
valencia_job.result().get_counts()

In [ ]:
job_monitor(job)

In [78]:
backend.defaults().meas_freq_est

[4.744512967823727e+18,
 4.650699770187682e+18,
 4.792321136865513e+18,
 4.834117996817102e+18,
 4.959362381758176e+18]